[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/trainer/blob/main/scenedetect.ipynb)

In [ ]:
!pip install -q yt_dlp scenedetect[opencv]

In [ ]:
from yt_dlp import YoutubeDL

def download_video(id):
  ydl_opts = {'overwrites':True, 'format':'bestvideo[ext=mp4]+bestaudio[ext=m4a]/mp4', 'outtmpl':f'/content/videos/{id}.mp4'}
  with YoutubeDL(ydl_opts) as ydl:
    ydl.download(id)
    return f"/content/videos/{id}.mp4"

In [ ]:
video_id = 'TnGl01FkMMo'
download_video(video_id)
video_path = f"/content/videos/{video_id}.mp4"
# !scenedetect -i {video_path} -o /content/adaptive --drop-short-scenes detect-adaptive save-images
!scenedetect -i {video_path} -o /content/content --drop-short-scenes detect-content save-images
# !scenedetect -i {video_path} -o /content/threshold --drop-short-scenes detect-threshold save-images

In [ ]:
%cd /content/content
!zip -r content.zip ./